In [1]:
import pandas as pd
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
from langchain.chains import LLMChain
import json
from langchain.prompts import (
    ChatPromptTemplate,
)
from langchain.llms import OpenAI




In [2]:
from langfuse.callback import CallbackHandler
handler = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [3]:
GENERATIONS_PATH = "../datasets/error_analysis/zero_shot_cot_50_50_merged.csv"
MODEL1 = "gpt-3.5-turbo-0301"
MODEL2 = "gpt-3.5-turbo-0301"

In [4]:
data = pd.read_csv(GENERATIONS_PATH)




if "similarity" not in data.columns:
    data["similarity"] = ""
if "confidence" not in data.columns:
    data["confidence"] = ""

data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,similarity,confidence
0,0,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:57:21.132Z,2023-11-03T16:57:35.726Z,Are the following two products the same?\nProd...,1. Both products are related to Traveler's Com...,No match.,False,True,FN,,
1,1,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:56:59.632Z,2023-11-03T16:57:19.296Z,Are the following two products the same?\nProd...,Step 1: Title\nBoth products have the same tit...,Match.,True,True,NONE,,
2,2,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:56:28.065Z,2023-11-03T16:56:57.281Z,Are the following two products the same?\nProd...,1. Both products are related to the Traveler's...,Match.,True,True,NONE,,
3,3,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:56:08.243Z,2023-11-03T16:56:24.431Z,Are the following two products the same?\nProd...,1. Both products are related to Traveler's not...,Match.,True,True,NONE,,
4,4,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:55:47.096Z,2023-11-03T16:56:03.552Z,Are the following two products the same?\nProd...,1. Both products have the same model number: C...,Match.,True,True,NONE,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:04:31.389Z,2023-11-03T15:04:47.193Z,Are the following two products the same?\nProd...,Step 1: Compare the titles\nThe titles of the ...,No match.,False,False,NONE,,
217,217,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:03:59.177Z,2023-11-03T15:04:29.889Z,Are the following two products the same?\nProd...,Step 1: Look at the titles\nProduct 1: Instax ...,No match.,False,False,NONE,,
218,218,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:03:45.755Z,2023-11-03T15:03:56.505Z,Are the following two products the same?\nProd...,Step 1: Compare the titles\nThe titles of the ...,no match,False,False,NONE,,
219,219,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:03:22.941Z,2023-11-03T15:03:43.606Z,Are the following two products the same?\nProd...,Step 1: Compare the titles\nProduct 1: Instax ...,no match,False,False,NONE,,


In [5]:


task = """Now output your decision as a simple 'Match' or 'No Match'. 
Also provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the following format which includes an example output:
Decision: Match
Similarity: 75%
Confidence: 60%
"""

task2 = """Now provide a similarity score for the items and a confidence score for your decision as percentage values, 100% referring to perfect similarity or full confidence. 
Please provide your answer exactly in the same format as the following example output:
Similarity: 75%
Confidence: 60%
"""


STEP_CONFIDENCE_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", task2),
        ])





In [6]:
def get_confidence_score(start, end):
    llm = ChatOpenAI(model_name=MODEL1, temperature=0, max_tokens=150)
    chain = LLMChain(llm=llm, prompt=STEP_CONFIDENCE_PROMPT, callbacks=[handler])
    for i in range(start, end):
        print(i)
        whole_prompt = data["prompt"][i]
        user_prompt = "Q:" + whole_prompt.split("Q:")[-1]
        ai_answer = data["completion"][i]
        if data["predicted_label"][i]:
            prediction = "Match"
        else:
            prediction = "No Match"
        ai_answer = ai_answer + "\n" + prediction
        try:
            output = chain.run(user_prompt=user_prompt, ai_answer=ai_answer, callbacks=[handler])
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_CONFIDENCE_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer))
        print("----")
        print(data["predicted_label"][i])
        print(output)
        print("-----------")
        splitted_output = output.split("\n")
        # decision = splitted_output[0].split(": ")[1]
        similarity = splitted_output[0].split(": ")[1]
        confidence = splitted_output[1].split(": ")[1]
        # data["decision"][i] = decision
        data["similarity"][i] = similarity
        data["confidence"][i] = confidence



In [7]:
# get_confidence_score(0, len(data))
# data.to_csv(GENERATIONS_PATH, index=False)


0
System: You are a helpful AI.
Human: Q:Are the following two products the same?
Product 1: Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2: Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none

Lets think step by step!
AI: 1. Both products are related to Traveler's Company notebooks.\n2. Product 1 is specifically a refill for a regular size lined notebook.\n3. Product 1 has a description that mentions the interval of the lines and the type of paper used.\n4. Product 2 has no description provided.\n5. Without more information, it is unclear if Product 2 is the same as Product 1 or if it is a different accessory for Traveler's Company notebooks.
No Match
Human: Now provide a simi

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence


System: You are a helpful AI.
Human: Q:Are the following two products the same?
Product 1: Title: MINION DHR II 29 X 2.30 3C TERRA TR DD FOLD
Description: Refined and redesigned; ready to shred any lineRamped and siped tread to roll fast and track straight under brakingSingle-ply, DoubleDown, and DH casings available3C MaxxTerra, 3C MaxxGrip, and Super Tacky available on select modelsWide trail (WT) ca
Product 2: Title: Maxxis Minion DHR II 29\" DD/3C/TR
Description: Minion DHR II 29 x 2.35 tubeless ready tyre with Double down casing and 3C Maxx Terra.For our full range of Bike Accessories visit Evans Cycles

Lets think step by step!
AI: Step 1: Title\nBoth products have the same title, \Minion DHR II 29\.\n\nStep 2: Description\nBoth products describe the same tire, the Minion DHR II 29, with similar features such as a tubeless ready design, DoubleDown casing, and 3C MaxxTerra technology. However, Product 1 provides more detailed information about the tire's tread design and casing op

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence


System: You are a helpful AI.
Human: Q:Are the following two products the same?
Product 1: Title: Traveler's Company 014. CRAFT PAPER (64PG) MIDORI TRAVELER'S NOTEBOOK
Description: none
Product 2: Title: Refill #014: Kraft Paper Notebook
Description: Cardboard Paper notebook refill for the Traveler's Notebook.Various cuttings, stickers, photos, scribbles and more sophisticated drawings suit it well.Contains 64 pages and its dimensions (210 x 110 mm) make it suitable for the classic version, not f

Lets think step by step!
AI: 1. Both products are related to the Traveler's Notebook.\n2. Product 1 is titled \Traveler's Company 014. CRAFT PAPER (64PG) MIDORI TRAVELER'S NOTEBOOK\ and has no description.\n3. Product 2 is titled \Refill #014: Kraft Paper Notebook\ and has a description that includes the fact that it is a cardboard paper notebook refill for the Traveler's Notebook, contains 64 pages, and has dimensions of 210 x 110 mm.\n4. Product 2 also mentions that it is suitable for the c

...

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence


System: You are a helpful AI.
Human: Q:Are the following two products the same?
Product 1: Title: Instax Mini Rainbow Film 10 Pack
Description: Brighten the place up with this pack of rainbow coloured film! Top tip: one of these would look fab in our Instax Mini Acrylic Photo Block!Includes pack of 10 shots
Product 2: Title: 1 Fujifilm Instax Square Film Black Frame
Description: none

Lets think step by step!
AI: Step 1: Compare the titles\nProduct 1: Instax Mini Rainbow Film 10 Pack\nProduct 2: Fujifilm Instax Square Film Black Frame\n\nThe titles are different, so the products are likely not the same.\n\nStep 2: Compare the descriptions\nProduct 1: Rainbow coloured film, pack of 10 shots, recommended for use with Instax Mini Acrylic Photo Block\nProduct 2: No description provided\n\nThe descriptions are different, and Product 2 has no description, so the products are likely not the same.\n\nConclusion: Based on the title and description, it appears that Product 1 is a pack of rainbow

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence


System: You are a helpful AI.
Human: Q:Are the following two products the same?
Product 1: Title: DJI Mavic Mini Drone Fly More Combo
Description: Mavic Mini, the everyday flycam. Open your sky with the compact, yet powerful Mavic Mini. Weighing under 250 grams, together with the DJI Fly APP, you'll enjoy a simplified flying experience that is almost as light as the average smartphone.
Product 2: Title: Fujifilm Instax Mini Instant Film - Monochrome (10 Shots)
Description: Monochrome (Black & White) Instant Film for Fujifilm Mini Instant Cameras10 Shots per PackISO 800High Gloss FinishSuperb Grain QualityFast Image FormationGreater Resistance to Age-Induced ChangesSingle-plastic Composition for Easier Disposal

Lets think step by step!
AI: Step 1: Are the titles the same?\nNo, the titles are different.\n\nStep 2: Are the descriptions related to the same product?\nNo, the descriptions are for two different products. \n\nTherefore, the two products are not the same.
No Match
Human: Now p

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["similarity"][i] = similarity
/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_20047/2477896871.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["confidence"][i] = confidence


In [8]:
data


,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,similarity,confidence
0,0,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:57:21.132Z,2023-11-03T16:57:35.726Z,Are the following two products the same?\nProd...,1. Both products are related to Traveler's Com...,No match.,False,True,FN,50%,40%
1,1,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:56:59.632Z,2023-11-03T16:57:19.296Z,Are the following two products the same?\nProd...,Step 1: Title\nBoth products have the same tit...,Match.,True,True,NONE,90%,80%
2,2,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:56:28.065Z,2023-11-03T16:56:57.281Z,Are the following two products the same?\nProd...,1. Both products are related to the Traveler's...,Match.,True,True,NONE,60%,80%
3,3,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:56:08.243Z,2023-11-03T16:56:24.431Z,Are the following two products the same?\nProd...,1. Both products are related to Traveler's not...,Match.,True,True,NONE,60%,70%
4,4,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T16:55:47.096Z,2023-11-03T16:56:03.552Z,Are the following two products the same?\nProd...,1. Both products have the same model number: C...,Match.,True,True,NONE,90%,80%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:04:31.389Z,2023-11-03T15:04:47.193Z,Are the following two products the same?\nProd...,Step 1: Compare the titles\nThe titles of the ...,No match.,False,False,NONE,0%,100%
217,217,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:03:59.177Z,2023-11-03T15:04:29.889Z,Are the following two products the same?\nProd...,Step 1: Look at the titles\nProduct 1: Instax ...,No match.,False,False,NONE,20%,80%
218,218,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:03:45.755Z,2023-11-03T15:03:56.505Z,Are the following two products the same?\nProd...,Step 1: Compare the titles\nThe titles of the ...,no match,False,False,NONE,0%,100%
219,219,6eb5b386-ec6a-47c5-885a-0503f80bf6e4,gpt-3.5-turbo-0301,2023-11-03T15:03:22.941Z,2023-11-03T15:03:43.606Z,Are the following two products the same?\nProd...,Step 1: Compare the titles\nProduct 1: Instax ...,no match,False,False,NONE,0%,90%


In [9]:
task_structured_explanation = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95" "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00" "values":["missing","Graphite"]}]"""

task_structured_explanation_v2 = """Explain your decision in a structured format, listing the attributes that you compared for reaching your decision. Each attribute should be accompanied by the attribute values and a score between 0 and 1 that shows the importance of the attribute for the decision. If an attribute only occurs in one item, specify the value of that attribute for the other item as "missing". The attributes should not be just the Title or Description but more specific attributes like Brand, Model, Color, etc. 
The format should exactly match the following example output: 
[{"attribute":"brand","importance":"0.05","values":["Logitech","Logitech"]},{"attribute":"model","importance":"0.95", "values":["G500","MX Master 3S"]},{"attribute":"color","importance":"0.00", "values":["missing","Graphite"]}]"""



STEP_EXPLANATION_STRUCTURED_PROMPT = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful AI."),
            ("human", "{user_prompt}"),
            ("ai", "{ai_answer}"),
            ("human", "{task_structured_explanation}"),
        ])




In [10]:
# if "structured_explanation" not in data.columns:
    # data["structured_explanation"] = ""
if "structured_explanation_v2" not in data.columns:
    data["structured_explanation_v2"] = ""

In [11]:
handler2 = CallbackHandler(os.environ.get("LF_PK"), os.environ.get("LF_SK"), os.environ.get("LF_HOST"))

In [12]:
def get_structured_explanation(start, end):
    if (MODEL2 == "gpt-3.5-turbo-instruct"):
        llm = OpenAI(model_name=MODEL2, temperature=0)
    else:
        llm = ChatOpenAI(model_name=MODEL2, temperature=0)
    chain = LLMChain(llm=llm, prompt=STEP_EXPLANATION_STRUCTURED_PROMPT, callbacks=[handler2])
    for i in range(start, end):
        print(i)
        whole_prompt = data["prompt"][i]
        user_prompt = "Q:" + whole_prompt.split("Q:")[-1]
        ai_answer = data["completion"][i]
        if data["predicted_label"][i]:
            prediction = "Match"
        else:
            prediction = "No Match"
        ai_answer = ai_answer + "\n" + prediction
        try:
            output = chain.run(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2, callbacks=[handler2])
        except:
            output = "error"
            i = i - 1
            continue
        print(STEP_EXPLANATION_STRUCTURED_PROMPT.format(user_prompt=user_prompt, ai_answer=ai_answer, task_structured_explanation=task_structured_explanation_v2))
        print("----")
        print(data["label"][i])
        print(output)
        print("-----------")
        data["structured_explanation_v2"][i] = output



In [13]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,similarity,confidence,structured_explanation_v2
0,0,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:55.032Z,2023-10-20T12:40:03.460Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,True,FN,20%,90%,
1,1,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:42.734Z,2023-10-20T12:39:53.607Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",The two products may match.,True,True,NONE,90%,80%,
2,2,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:28.325Z,2023-10-20T12:39:40.801Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,90%,80%,
3,3,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:39:10.835Z,2023-10-20T12:39:26.196Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",Match.,True,True,NONE,60%,80%,
4,4,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T12:38:55.776Z,2023-10-20T12:39:08.718Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles of the two pr...",The two products may match.,True,True,NONE,90%,80%,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:42.555Z,2023-10-20T11:59:51.404Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,0%,100%,
217,217,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:28.660Z,2023-10-20T11:59:41.048Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,25%,90%,
218,218,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:17.151Z,2023-10-20T11:59:26.845Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,0%,100%,
219,219,136e98ba-ac14-42f4-8345-103a04fd95e3,gpt-3.5-turbo-0301,2023-10-20T11:59:04.839Z,2023-10-20T11:59:15.798Z,Q: Are the following two products the same?\nP...,"First, we can compare the titles and descripti...",no match,False,False,NONE,25%,80%,


In [14]:
get_structured_explanation(0, len(data))

0
System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1:
Title: Traveler's Lined Notebook Refill - Regular Size
Description: Think about the days you went to school. Based on people’s past experience as a student, we receive comments saying that this is the most easy type of notebook to use. The lines are printed with an interval of 6.5mm, and the paper is our originally m
Product 2:
Title: Traveler's Company Traveler's Company Notebook Accessories
Description: none
A: Let's think step by step!
AI: First, we can compare the titles and descriptions of the two products. The first product is a refill for a lined notebook in regular size, with a specific interval between the lines and made by a specific company. The second product is a set of notebook accessories made by the same company, but with no specific details mentioned. \nBased on this information, we can conclude that the two products are not the same. They are b

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_19946/3199974364.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2"][i] = output


System: You are a helpful AI.
Human: Q: Could you please tell me if the following two products are the same one?
Product 1:
Title: MINION DHR II 29 X 2.30 3C TERRA TR DD FOLD
Description: Refined and redesigned; ready to shred any lineRamped and siped tread to roll fast and track straight under brakingSingle-ply, DoubleDown, and DH casings available3C MaxxTerra, 3C MaxxGrip, and Super Tacky available on select modelsWide trail (WT) ca
Product 2:
Title: Maxxis Minion DHR II 29\" DD/3C/TR
Description: Minion DHR II 29 x 2.35 tubeless ready tyre with Double down casing and 3C Maxx Terra.For our full range of Bike Accessories visit Evans Cycles
A: Let's think step by step!
AI: First, we can compare the titles and descriptions of the two products. Both products are Maxxis Minion DHR II 29\ tires, but there are some differences in the details. \nProduct 1 mentions a 2.30 size, 3C Terra TR DD Fold, and different casing options. Product 2 mentions a 2.35 size, DD/3C/TR, and tubeless readiness.

/var/folders/gx/tzjwcf8x1llbhhf36mjt7xqr0000gn/T/ipykernel_19946/3199974364.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["structured_explanation_v2"][i] = output


3


In [ ]:
data

,Unnamed: 0,traceId,model,startTime,endTime,prompt,completion,eval,predicted_label,label,error,decision,similarity,confidence,structured_explanation,structured_explanation_v2_instruct
0,0,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:22.070Z,2023-10-17T10:25:25.422Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,True,FN,No Match,0%,100%,"[\n {\n ""attribute"": ""Title"",\n ""import...","\n\n[{""attribute"":""title"",""importance"":""0.5"",""..."
1,1,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:12.823Z,2023-10-17T10:25:17.121Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same one. They...",Match.,True,True,NONE,Match,95%,90%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
2,2,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:25:04.230Z,2023-10-17T10:25:07.905Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same. They are...",Match.,True,True,NONE,Match,100%,95%,"[{""attribute"":""Product Number"",""importance"":""0...","\n\n[{""attribute"":""Brand"",""importance"":""0.05"",..."
3,3,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:56.337Z,2023-10-17T10:24:59.610Z,Q: Could you please tell me if the following t...,These two products are likely the same because...,Match.,True,True,NONE,Match,90%,80%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
4,4,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T10:24:48.374Z,2023-10-17T10:24:51.714Z,Q: Could you please tell me if the following t...,"Yes, these two products are the same one. They...",Match.,True,True,NONE,Match,100%,95%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,216,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:58.234Z,2023-10-17T09:52:02.222Z,Q: Could you please tell me if the following t...,These two products are not the same one. The f...,No match.,False,False,NONE,No Match,0%,100%,Here are the attributes that I compared for re...,"\n\n[{""attribute"":""brand"",""importance"":""0.5"",""..."
217,217,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:46.656Z,2023-10-17T09:51:54.143Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""Brand"",\n ""import...","\n\n[{""attribute"":""brand"",""importance"":""0.1"",""..."
218,218,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:37.814Z,2023-10-17T09:51:42.110Z,Q: Could you please tell me if the following t...,These products are not the same one as they ar...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""category"",\n ""imp...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."
219,219,c58f0c2d-d624-44b6-a73e-900e8ac46026,gpt-3.5-turbo-0301,2023-10-17T09:51:27.877Z,2023-10-17T09:51:33.410Z,Q: Could you please tell me if the following t...,These two products are not the same because th...,No match.,False,False,NONE,No Match,0%,100%,"[\n {\n ""attribute"": ""Camera Type"",\n ""...","\n\n[{""attribute"":""brand"",""importance"":""0.05"",..."


In [ ]:
data.to_csv(GENERATIONS_PATH, index=False)
